# Big data mining and Applications  Course
## HW#1 - Individual household electric power consumption Data Set
### Data Set Information:
Available at: https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption 

[Individual household electric power consumption dataset] from UCI Machine Learning Repository About 2 million instances, 20MB (compressed) in size.

This archive contains 2075259 measurements gathered between December 2006 and November 2010 (47 months).
Notes:

1. (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.

2. The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.

### Attribute Information:
1. date: Date in format dd/mm/yyyy
2. time: time in format hh:mm:ss
3. global_active_power: household global minute-averaged active power (in kilowatt)
4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)
5. voltage: minute-averaged voltage (in volt)
6. global_intensity: household global minute-averaged current intensity (in ampere)
7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy).
    - It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).
8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). 
    - It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.
9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy).
    - It corresponds to an electric water-heater and an air-conditioner.

### Check Hardware Specification 

In [93]:
%cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
stepping	: 2
microcode	: 0x35
cpu MHz		: 2397.223
cache size	: 15360 KB
physical id	: 0
siblings	: 1
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts mmx fxsr sse sse2 ss syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts nopl xtopology tsc_reliable nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm cpuid_fault invpcid_single pti retpoline fsgsbase tsc_adjust bmi1 avx2 smep bmi2 invpcid xsaveopt arat
bugs		: cpu_meltdown spectre_v1 spectre_v2
bogomips	: 4794.44
clflush size	: 64
cache_alignment	: 64
address sizes	: 43 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: Gen

In [94]:
%cat /proc/meminfo

MemTotal:        8168740 kB
MemFree:          140652 kB
MemAvailable:     515880 kB
Buffers:          145764 kB
Cached:           516752 kB
SwapCached:        13776 kB
Active:          6303360 kB
Inactive:        1410664 kB
Active(anon):    6010656 kB
Inactive(anon):  1142228 kB
Active(file):     292704 kB
Inactive(file):   268436 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:       8385532 kB
SwapFree:        8302064 kB
Dirty:               192 kB
Writeback:             0 kB
AnonPages:       7038768 kB
Mapped:           380384 kB
Shmem:            101368 kB
Slab:             150720 kB
SReclaimable:      70992 kB
SUnreclaim:        79728 kB
KernelStack:       27488 kB
PageTables:        57764 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    12469900 kB
Committed_AS:   16696048 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
HardwareCorrupted:     0 kB
AnonHugePages:   

### Check Operating System Specification

In [95]:
%cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=16.04
DISTRIB_CODENAME=xenial
DISTRIB_DESCRIPTION="Ubuntu 16.04.4 LTS"


In [96]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import rand, randn

In [97]:
conf = SparkConf().setMaster("spark://sparklab1:7077").setAppName("Household Power Consumption")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext=SQLContext(sc)

### Check Apache Spark environtment

In [98]:
sc

<SparkContext master=spark://sparklab1:7077 appName=PySparkShell>

In [99]:
raw_data = sc.textFile("hdfs://tri-virtual-machine:9000/user/tri/household_power_consumption.txt",use_unicode=False)

In [100]:
df = raw_data.map(lambda x: x.split(";"))

In [101]:
df.take(5)

[['Date',
  'Time',
  'Global_active_power',
  'Global_reactive_power',
  'Voltage',
  'Global_intensity',
  'Sub_metering_1',
  'Sub_metering_2',
  'Sub_metering_3'],
 ['16/12/2006',
  '17:24:00',
  '4.216',
  '0.418',
  '234.840',
  '18.400',
  '0.000',
  '1.000',
  '17.000'],
 ['16/12/2006',
  '17:25:00',
  '5.360',
  '0.436',
  '233.630',
  '23.000',
  '0.000',
  '1.000',
  '16.000'],
 ['16/12/2006',
  '17:26:00',
  '5.374',
  '0.498',
  '233.290',
  '23.000',
  '0.000',
  '2.000',
  '17.000'],
 ['16/12/2006',
  '17:27:00',
  '5.388',
  '0.502',
  '233.740',
  '23.000',
  '0.000',
  '1.000',
  '17.000']]

In [102]:
df2 = sqlContext.createDataFrame(data = df.filter(lambda x:x[0]!='Date'),
                                schema = df.filter(lambda x:x[0]=='Date').collect()[0])
df2.persist()

DataFrame[Date: string, Time: string, Global_active_power: string, Global_reactive_power: string, Voltage: string, Global_intensity: string, Sub_metering_1: string, Sub_metering_2: string, Sub_metering_3: string]

In [103]:
df2.show(n = 20)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418|234.840|          18.400|         0.000|         1.000|        17.000|
|16/12/2006|17:25:00|              5.360|                0.436|233.630|          23.000|         0.000|         1.000|        16.000|
|16/12/2006|17:26:00|              5.374|                0.498|233.290|          23.000|         0.000|         2.000|        17.000|
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|        17.000|
|16/12/2006|17:28:00|              3.666|                0.528

In [104]:
#check the types of each columns
df2.dtypes

[('Date', 'string'),
 ('Time', 'string'),
 ('Global_active_power', 'string'),
 ('Global_reactive_power', 'string'),
 ('Voltage', 'string'),
 ('Global_intensity', 'string'),
 ('Sub_metering_1', 'string'),
 ('Sub_metering_2', 'string'),
 ('Sub_metering_3', 'string')]

In [105]:
from pyspark.sql.types import IntegerType, DateType, DoubleType

In [106]:
#Change String types format to DoubleType "Global_active_power","Global_reactive_power","Voltage","Global_intensity"
df3 = df2.withColumn("Global_active_power", df2["Global_active_power"].cast(DoubleType()))
df4 = df3.withColumn("Global_reactive_power", df3["Global_reactive_power"].cast(DoubleType()))
df5 = df4.withColumn("Voltage", df4["Voltage"].cast(DoubleType()))
df6 = df5.withColumn("Global_intensity", df5["Global_intensity"].cast(DoubleType()))


#### Check datafreame types

In [107]:
df6.dtypes

[('Date', 'string'),
 ('Time', 'string'),
 ('Global_active_power', 'double'),
 ('Global_reactive_power', 'double'),
 ('Voltage', 'double'),
 ('Global_intensity', 'double'),
 ('Sub_metering_1', 'string'),
 ('Sub_metering_2', 'string'),
 ('Sub_metering_3', 'string')]

##### Check schema dataframe

In [108]:
df6.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Global_active_power: double (nullable = true)
 |-- Global_reactive_power: double (nullable = true)
 |-- Voltage: double (nullable = true)
 |-- Global_intensity: double (nullable = true)
 |-- Sub_metering_1: string (nullable = true)
 |-- Sub_metering_2: string (nullable = true)
 |-- Sub_metering_3: string (nullable = true)



In [109]:
df6.show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418| 234.84|            18.4|         0.000|         1.000|        17.000|
|16/12/2006|17:25:00|               5.36|                0.436| 233.63|            23.0|         0.000|         1.000|        16.000|
|16/12/2006|17:26:00|              5.374|                0.498| 233.29|            23.0|         0.000|         2.000|        17.000|
|16/12/2006|17:27:00|              5.388|                0.502| 233.74|            23.0|         0.000|         1.000|        17.000|
|16/12/2006|17:28:00|              3.666|                0.528

In [110]:
df6.describe(["Global_active_power","Global_reactive_power","Voltage","Global_intensity"]).show()

+-------+-------------------+---------------------+------------------+-----------------+
|summary|Global_active_power|Global_reactive_power|           Voltage| Global_intensity|
+-------+-------------------+---------------------+------------------+-----------------+
|  count|            2049280|              2049280|           2049280|          2049280|
|   mean| 1.0916150365007313|  0.12371447630388727|240.83985797455423|4.627759310588489|
| stddev|  1.057294161093981|   0.1127219795507159|3.2399866790102716|4.444396259786191|
|    min|              0.076|                  0.0|             223.2|              0.2|
|    max|             11.122|                 1.39|            254.15|             48.4|
+-------+-------------------+---------------------+------------------+-----------------+



#### Using library of pyspark to anaysis minimum, maximum, mean and standar deviation

In [111]:
from pyspark.sql.functions import min,max,mean,stddev,count

#### Output the minimum, maximum and count of the columns 'Global_active_power'

In [112]:
print "Show output the minimum, maximum and count of the columns 'Global_active_power'"
df6.select([min("Global_active_power").alias("Minimum Global Active Power"),
           max("Global_active_power").alias("Maximum Global Active Power"),
           count("Global_active_power").alias("Count Global Active Power")]).show()

Show output the minimum, maximum and count of the columns 'Global_active_power'
+---------------------------+---------------------------+-------------------------+
|Minimum Global Active Power|Maximum Global Active Power|Count Global Active Power|
+---------------------------+---------------------------+-------------------------+
|                      0.076|                     11.122|                  2049280|
+---------------------------+---------------------------+-------------------------+



In [113]:
print "Count each value of Global Active Power"
df6.groupBy("Global_active_power").count().show(n = 30)

Count each value of Global Active Power
+-------------------+-----+
|Global_active_power|count|
+-------------------+-----+
|               3.26|  163|
|              2.784|  273|
|              3.456|  147|
|              1.462| 2310|
|               0.66|  802|
|              2.712|  325|
|              4.132|   86|
|              0.204| 4846|
|              1.882|  667|
|              7.818|    3|
|              0.526| 1431|
|              0.194| 2748|
|              2.952|  248|
|              2.132|  441|
|               2.86|  238|
|              4.318|   82|
|              4.592|   33|
|              4.752|   42|
|              10.65|    1|
|              5.114|   31|
|              0.134| 1394|
|              2.206|  412|
|              3.174|  176|
|              0.262| 6188|
|               1.82|  690|
|              2.802|  252|
|              1.246|  938|
|              1.642| 1053|
|              0.772|  609|
|              2.808|  260|
+-------------------+-----+
only sho

#### Output the minimum, maximum and count of the columns 'Global_reactive_power'

In [114]:
print "Show output the minimum, maximum and count of the columns 'Global_reactive_power'"
df6.select([min("Global_reactive_power").alias("Minimum Global Reactive Power"),
           max("Global_reactive_power").alias("Maximum Global Reactive Power"),
           count("Global_reactive_power").alias("Count Global Reactive Power")]).show()

Show output the minimum, maximum and count of the columns 'Global_reactive_power'
+-----------------------------+-----------------------------+---------------------------+
|Minimum Global Reactive Power|Maximum Global Reactive Power|Count Global Reactive Power|
+-----------------------------+-----------------------------+---------------------------+
|                          0.0|                         1.39|                    2049280|
+-----------------------------+-----------------------------+---------------------------+



#### Output the minimum, maximum and count of the columns 'Voltage'

In [115]:
print "Show output the minimum, amximum and count of the columns 'Voltage'"
df6.select([min("Voltage").alias("Minimum Voltage"),
           max("Voltage").alias("Maximum Voltage"),
          count("Voltage").alias("Count Voltage")]).show()

Show output the minimum, amximum and count of the columns 'Voltage'
+---------------+---------------+-------------+
|Minimum Voltage|Maximum Voltage|Count Voltage|
+---------------+---------------+-------------+
|          223.2|         254.15|      2049280|
+---------------+---------------+-------------+



#### Output the minimum, maximum and count of the columns 'Global_intensity'

In [116]:
print "Show output the minimum, maximum and count the columns of 'Global_intensity'"
df6.select([min("Global_intensity").alias("Minimum Global Intensity "),
          max("Global_intensity").alias("Maximum Global Intensity"),
          count("Global_intensity").alias("Count Global Intensity")]).show()

Show output the minimum, maximum and count the columns of 'Global_intensity'
+-------------------------+------------------------+----------------------+
|Minimum Global Intensity |Maximum Global Intensity|Count Global Intensity|
+-------------------------+------------------------+----------------------+
|                      0.2|                    48.4|               2049280|
+-------------------------+------------------------+----------------------+



#### Output the mean and standar deviation 'Global_active_power'

In [117]:
print "Show the output mean and standar deviation of 'Global_active_power'"
df6.select([mean("Global_active_power").alias("Mean Global Active Power"),
           stddev("Global_active_power").alias("Standar Deviation Global Active Power")]).show()

Show the output mean and standar deviation of 'Global_active_power'
+------------------------+-------------------------------------+
|Mean Global Active Power|Standar Deviation Global Active Power|
+------------------------+-------------------------------------+
|      1.0916150365007313|                    1.057294161093981|
+------------------------+-------------------------------------+



#### Output the mean and standar deviation 'Global_reactive_power'

In [118]:
print "Show the output mean and standar deviation of  'Global_reactive_power'"
df6.select([mean("Global_reactive_power").alias("Mean Global Reactive Power"),
           stddev("Global_reactive_power").alias("Standar Global Reactive power")]).show()

Show the output mean and standar deviation of  'Global_reactive_power'
+--------------------------+-----------------------------+
|Mean Global Reactive Power|Standar Global Reactive power|
+--------------------------+-----------------------------+
|       0.12371447630388727|           0.1127219795507159|
+--------------------------+-----------------------------+



#### Output the mean and standar deviation 'Voltage'

In [119]:
print "Show the output mean and standar deviation of 'Voltage'"
df6.select([mean("Voltage").alias("Mean Voltage"),
           stddev("Voltage").alias("Standar Deviation Voltage")]).show()

Show the output mean and standar deviation of 'Voltage'
+------------------+-------------------------+
|      Mean Voltage|Standar Deviation Voltage|
+------------------+-------------------------+
|240.83985797455423|       3.2399866790102716|
+------------------+-------------------------+



#### Output the mean and standar deviation 'Global_intensity'

In [120]:
print "Show the output mean and standar deviation of 'Global_intensity'"
df6.select([mean("Global_intensity").alias("Mean Global Intensity"),
           stddev("Global_intensity").alias("Standar Deviation Global Intensity")]).show()

Show the output mean and standar deviation of 'Global_intensity'
+---------------------+----------------------------------+
|Mean Global Intensity|Standar Deviation Global Intensity|
+---------------------+----------------------------------+
|    4.627759310588489|                 4.444396259786191|
+---------------------+----------------------------------+



In [121]:
from pyspark.sql import functions as F
df6.groupBy("Date","Time").agg(F.sum("Global_active_power").alias("Global Active Power"),
                       F.sum("Global_reactive_power").alias("Global Reactive Power"),
                       F.sum("Voltage").alias("Voltage"),
                       F.sum("Global_intensity").alias("Global Intensity")).show(n = 30)

+----------+--------+-------------------+---------------------+-------+----------------+
|      Date|    Time|Global Active Power|Global Reactive Power|Voltage|Global Intensity|
+----------+--------+-------------------+---------------------+-------+----------------+
|16/12/2006|19:59:00|              3.214|                0.078| 232.66|            13.8|
|16/12/2006|20:27:00|              3.258|                0.076| 234.57|            13.8|
|16/12/2006|21:40:00|               2.36|                0.064| 236.89|            10.8|
|17/12/2006|03:19:00|              0.424|                  0.0| 244.88|             3.2|
|17/12/2006|04:39:00|              3.412|                0.052| 242.49|            14.0|
|17/12/2006|09:44:00|              0.338|                0.076| 241.21|             1.4|
|17/12/2006|13:07:00|              1.712|                0.344| 236.85|             7.2|
|17/12/2006|14:37:00|              2.118|                 0.25| 242.89|             8.6|
|17/12/2006|14:53:00|

### (3) Perform min-max normalization on the columns to generate normalized output

The min-max technique is based on the  mmin and amx values of the attribute as follows.
Normalize values will be between 0 and 1

Vn = (v - vmin) / (vmax - vmin) where

vn = normalized value
<br>v = original value
<br>vmin = minimum value
<br>vmax = maximum value

In [122]:
from pyspark.sql.functions import col
(df6.select(min("Global_active_power").alias("MIN_Global_active_power"),
            max("Global_active_power").alias("MAX_Global_active_power")).crossJoin(df6).withColumn("Min-Max_Normalization",(col("Global_active_power") - col("MIN_Global_active_power")) / (col("MAX_Global_active_power") - col("MIN_Global_active_power")))).select("Global_active_power",
            "MIN_Global_active_power",
            "MAX_Global_active_power",
            "Min-Max_Normalization").show()

+-------------------+-----------------------+-----------------------+---------------------+
|Global_active_power|MIN_Global_active_power|MAX_Global_active_power|Min-Max_Normalization|
+-------------------+-----------------------+-----------------------+---------------------+
|              4.216|                  0.076|                 11.122|   0.3747963063552418|
|               5.36|                  0.076|                 11.122|   0.4783632084012313|
|              5.374|                  0.076|                 11.122|   0.4796306355241717|
|              5.388|                  0.076|                 11.122|  0.48089806264711216|
|              3.666|                  0.076|                 11.122|   0.3250045265254391|
|               3.52|                  0.076|                 11.122|    0.311787072243346|
|              3.702|                  0.076|                 11.122|   0.3282636248415716|
|                3.7|                  0.076|                 11.122|  0.3280825

In [123]:
(df6.select(min("Global_reactive_power").alias("MIN_Global_Reactive_power"),
            max("Global_reactive_power").alias("MAX_Global_Reactive_power")).crossJoin(df6).withColumn("Min-Max_Normalization",(col("Global_reactive_power") - col("MIN_Global_Reactive_power")) / (col("MAX_Global_Reactive_power") - col("MIN_Global_Reactive_power")))).select("Global_Reactive_power",
            "MIN_Global_Reactive_power",
            "MAX_Global_Reactive_power",
            "Min-Max_Normalization").show()

+---------------------+-------------------------+-------------------------+---------------------+
|Global_Reactive_power|MIN_Global_Reactive_power|MAX_Global_Reactive_power|Min-Max_Normalization|
+---------------------+-------------------------+-------------------------+---------------------+
|                0.418|                      0.0|                     1.39|  0.30071942446043165|
|                0.436|                      0.0|                     1.39|  0.31366906474820144|
|                0.498|                      0.0|                     1.39|  0.35827338129496406|
|                0.502|                      0.0|                     1.39|   0.3611510791366907|
|                0.528|                      0.0|                     1.39|   0.3798561151079137|
|                0.522|                      0.0|                     1.39|  0.37553956834532376|
|                 0.52|                      0.0|                     1.39|   0.3741007194244605|
|                 0.

In [124]:
(df6.select(min("Voltage").alias("MIN_Voltage"),
            max("Voltage").alias("MAX_Voltage")).crossJoin(df6).withColumn("Min-Max_Normalization",(col("Voltage") - col("MIN_Voltage")) / (col("MAX_Voltage") - col("MIN_Voltage")))).select("Voltage",
            "MIN_Voltage",
            "MAX_Voltage",
            "Min-Max_Normalization").show()

+-------+-----------+-----------+---------------------+
|Voltage|MIN_Voltage|MAX_Voltage|Min-Max_Normalization|
+-------+-----------+-----------+---------------------+
| 234.84|      223.2|     254.15|    0.376090468497577|
| 233.63|      223.2|     254.15|  0.33699515347334413|
| 233.29|      223.2|     254.15|  0.32600969305331173|
| 233.74|      223.2|     254.15|   0.3405492730210021|
| 235.68|      223.2|     254.15|   0.4032310177705981|
| 235.02|      223.2|     254.15|   0.3819063004846531|
| 235.09|      223.2|     254.15|   0.3841680129240713|
| 235.22|      223.2|     254.15|  0.38836833602584825|
| 233.99|      223.2|     254.15|   0.3486268174474964|
| 233.86|      223.2|     254.15|  0.34442649434571954|
| 232.86|      223.2|     254.15|  0.31211631663974215|
| 232.78|      223.2|     254.15|  0.30953150242326355|
| 232.99|      223.2|     254.15|   0.3163166397415191|
| 232.91|      223.2|     254.15|   0.3137318255250405|
| 235.24|      223.2|     254.15|   0.3890145395

In [125]:
(df6.select(min("Global_intensity").alias("MIN_Global_intensity"),
            max("Global_intensity").alias("MAX_Global_intensity")).crossJoin(df6).withColumn("Min-Max_Normalization",(col("Global_reactive_power") - col("MIN_Global_Intensity")) / (col("MAX_Global_Intensity") - col("MIN_Global_Intensity")))).select("Global_intensity",
            "MIN_Global_Intensity",
            "MAX_Global_Intensity",
            "Min-Max_Normalization").show()

+----------------+--------------------+--------------------+---------------------+
|Global_intensity|MIN_Global_Intensity|MAX_Global_Intensity|Min-Max_Normalization|
+----------------+--------------------+--------------------+---------------------+
|            18.4|                 0.2|                48.4| 0.004522821576763486|
|            23.0|                 0.2|                48.4| 0.004896265560165976|
|            23.0|                 0.2|                48.4| 0.006182572614107884|
|            23.0|                 0.2|                48.4| 0.006265560165975104|
|            15.8|                 0.2|                48.4| 0.006804979253112034|
|            15.0|                 0.2|                48.4| 0.006680497925311204|
|            15.8|                 0.2|                48.4| 0.006639004149377594|
|            15.8|                 0.2|                48.4| 0.006639004149377594|
|            15.8|                 0.2|                48.4| 0.006431535269709544|
|   